<a href="https://colab.research.google.com/github/Brilliant-AI/20203799_LEEJUHO/blob/master/Run_example.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!git clone https://github.com/Brilliant-AI/20203799_LEEJUHO.git

Cloning into '20203799_LEEJUHO'...
remote: Enumerating objects: 66, done.
remote: Counting objects: 100% (66/66), done.
remote: Compressing objects: 100% (61/61), done.
remote: Total 66 (delta 18), reused 0 (delta 0), pack-reused 0
Unpacking objects: 100% (66/66), done.


In [2]:
cd '/content/20203799_LEEJUHO'

/content/20203799_LEEJUHO


In [3]:
!pip install -r requirements.txt

     |████████████████████████████████| 11.8MB 347kB/s 
ERROR: yellowbrick 0.9.1 has requirement scikit-learn>=0.20, but you'll have scikit-learn 0.18.1 which is incompatible.
ERROR: umap-learn 0.4.6 has requirement scikit-learn>=0.20, but you'll have scikit-learn 0.18.1 which is incompatible.
ERROR: imbalanced-learn 0.4.3 has requirement scikit-learn>=0.20, but you'll have scikit-learn 0.18.1 which is incompatible.
ERROR: fancyimpute 0.4.3 has requirement scikit-learn>=0.19.1, but you'll have scikit-learn 0.18.1 which is incompatible.
  Found existing installation: scikit-learn 0.22.2.post1
    Uninstalling scikit-learn-0.22.2.post1:
      Successfully uninstalled scikit-learn-0.22.2.post1


In [5]:
!tar xvzf DATA.tar.gz

DATA/iofrol.csv
DATA/paintcontrol.csv
DATA/gsdtsr.csv
DATA/


In [8]:
# run project

from run_experiment_common import *

def visualize():
    search_pattern = 'rq_*_stats.p'
    filename = 'rq'

    iteration_results = glob.glob(os.path.join(DATA_DIR, search_pattern))
    aggregated_results = os.path.join(DATA_DIR, filename)
    df = stats.load_stats_dataframe(iteration_results, aggregated_results)
    df.loc[pd.isnull(df['rewardfun']), 'rewardfun'] = 'none'
    mean_df = df[(df['detected'] + df['missed']) > 0].groupby(['step', 'env', 'agent', 'rewardfun'],
                                                              as_index=False).mean()

    # Single figures
    #

    for (agent, rewfun) in [('mlpclassifier', 'APHFW_reward'), ('tableau', 'timerank')]:
        for (agent, rewfun) in [('mlpclassifier', 'APHFW_reward')]:
            i = 0

            for env in sorted(mean_df['env'].unique(), reverse=True):
                plotname = 'rq2_napfd_abs_%s_%s_%s' % (env, agent, rewfun)
                print(env)

                reddf = mean_df[(mean_df['env'] == env) & (
                    mean_df['agent'].isin([agent, 'heur_sort', 'heur_weight', 'heur_random'])) & (
                                    mean_df['rewardfun'].isin([rewfun, 'none']))].groupby(['step', 'agent']).mean()[
                    'napfd'].unstack()
                # reddf['Heuristic Sort'] = reddf['heur_sort'] - reddf[agent]
                # reddf['Heuristic Weight'] = reddf['heur_weight'] - reddf[agent]
                # reddf['Random'] = reddf['heur_random'] - reddf[agent]
                # reddf['Random'] = reddf['heur_random']
                # reddf['Heuristic Sort'] = reddf['heur_sort']
                reddf['APHFW_reward'] = reddf[agent]
                del reddf[agent]
                del reddf['heur_sort']
                del reddf['heur_weight']
                del reddf['heur_random']
                window_size = 25
                r = reddf.groupby(reddf.index // window_size).mean()
                xdf = r.stack()
                xdf = xdf.reset_index(level=xdf.index.names)
                xdf['step'] *= window_size

                fig = plt.figure(figsize=(8, 4))
                ax = sns.barplot(data=xdf, x='step', y=0, hue='agent', figure=fig)

                ax.set_ylabel('')
                ax.set_xlabel('CI Cycle')
                ax.legend_.remove()

                if i == 0:
                    ax.set_ylabel('NAPFD')
                    # ax.set_ylim([-0.6, 0.6])
                    ax.set_title('Paint Control')
                    ax.legend(ncol=1, loc=1)
                elif i == 1:
                    ax.set_ylabel('NAPFD')
                    ax.set_title('IOF/ROL')
                    ax.legend(ncol=1, loc=1)
                elif i == 2 and len(mean_df['env'].unique()) == 3:
                    ax.set_ylabel('NAPFD')
                    ax.set_title('Google GSDTSR')
                    ax.legend(ncol=1, loc=1)

                ax.set_ylim([0, 1])
                # ax.set_ylim([-0.6, 0.6])
                fig.tight_layout()
                save_figures(fig, plotname)
                plt.clf()

                i += 1

run_experiments(exp_run_industrial_datasets, parallel=PARALLEL)

visualize()



/usr/local/lib/python3.6/dist-packages/sklearn/neural_network/multilayer_perceptron.py:912: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if np.setdiff1d(classes, self.classes_, assume_unique=True):
/usr/local/lib/python3.6/dist-packages/sklearn/neural_network/multilayer_perceptron.py:912: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if np.setdiff1d(classes, self.classes_, assume_unique=True):
/usr/local/lib/python3.6/dist-packages/sklearn/neural_network/multilayer_perceptron.py:912: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if np.setdiff1d(classes, self.class

Ran experiments: 1 results
paintcontrol
RESULTSrq2_napfd_abs_paintcontrol_mlpclassifier_APHFW_reward.pdf
iofrol
RESULTSrq2_napfd_abs_iofrol_mlpclassifier_APHFW_reward.pdf
gsdtsr
RESULTSrq2_napfd_abs_gsdtsr_mlpclassifier_APHFW_reward.pdf
paintcontrol
RESULTSrq2_napfd_abs_paintcontrol_mlpclassifier_APHFW_reward.pdf
iofrol
RESULTSrq2_napfd_abs_iofrol_mlpclassifier_APHFW_reward.pdf
gsdtsr
RESULTSrq2_napfd_abs_gsdtsr_mlpclassifier_APHFW_reward.pdf
